# Pointwise Evaluation

In this notebook, pointwise algorithms can be evaluated.
To start an experiment, define it using the following parameters:

<b>name</b>: Name of the experiment <br>
<b>model</b>: The model to use (Possible choices are nbg, lr, svm, dt, rf, ada, gb) <br>
<b>pca</b>: PCA components for dimensionality reduction (None with 0) <br>
<b>search_space</b>: Values to use in bayesian optimization (Optional) <br>
<b>trials</b>: Number of hyperparameter optimization trials (Optional)

### Imports

In [23]:
import os
import sys
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
sys.path.append(os.path.dirname((os.path.abspath(""))))

In [24]:
from src.pipeline import Pipeline

In [25]:
pipeline = Pipeline(
    collection='data/processed/30_5000_1000_collection.pkl',
    queries='data/processed/30_5000_1000_queries.pkl',
    queries_val='data/processed/30_5000_1000_queries_val.pkl',
    queries_test='data/processed/30_5000_1000_queries_test.pkl',
    features='data/processed/30_5000_1000_features.pkl',
    qrels_val='data/processed/30_5000_1000_qrels_val.pkl',
    qrels_test='data/processed/30_5000_1000_qrels_test.pkl',
    features_test='data/processed/30_5000_1000_features_test.pkl',
    features_val='data/processed/30_5000_1000_features_val.pkl',
)

<b>!</b> Run the next line only for feature selection

In [26]:
feature_selection_result = ['bert_cosine', 'bm25', 'char_rel_difference', 'bert_euclidean', 'w2v_cosine', 'w2v_tfidf_euclidean', 'polarity_doc', 'glove_cosine', 'query_verbs', 'doc_nouns']
pipeline.features = pipeline.features[['qID', 'pID', 'y'] + feature_selection_result]
pipeline.features_test = pipeline.features_test[['qID', 'pID'] + feature_selection_result]
pipeline.features_val = pipeline.features_val[['qID', 'pID'] + feature_selection_result]

In [27]:
pipeline.features

,qID,pID,y,bert_cosine,bm25,char_rel_difference,bert_euclidean,w2v_cosine,w2v_tfidf_euclidean,polarity_doc,glove_cosine,query_verbs,doc_nouns
0,603195,7050012,1,0.899372,-24.655536,10.269231,0.448616,0.972107,2.765727,0.000000,0.730834,1,23
1,474183,325505,1,0.880772,-33.129796,8.333333,0.488321,0.971866,1.360485,0.450000,0.864263,0,18
2,320545,1751825,1,0.582573,-16.699603,12.611111,0.913703,0.947701,2.236971,0.500000,0.858117,1,20
3,89798,5069949,1,0.731121,-27.678576,11.771429,0.733320,0.972710,1.714253,0.066667,0.645860,0,25
4,1054603,2869106,1,0.774794,-28.497519,10.928571,0.671127,0.965680,1.799412,0.000000,0.673893,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,128401,6127598,0,0.100652,-8.866170,13.666667,1.341154,0.796978,3.027138,-0.520833,0.051945,0,16
4996,1044540,4616118,0,0.476531,-7.852468,23.000000,1.023200,0.922095,2.167866,0.156250,0.003794,1,25
4997,486146,1137390,0,0.457971,-15.909103,5.875000,1.041181,0.946438,4.161341,-0.100000,0.787475,2,12
4998,532697,5161847,0,0.374877,-16.617979,9.250000,1.118144,0.938939,1.977307,0.284375,0.038626,0,18


## Random Baseline

In [5]:
pipeline.evaluate(
    name='default',
    model='baseline', 
    pca=0
)

MRR: 0.0019611375892770065
nDCG: 0.2380095700050317


## Logistic Regression

In [28]:
pipeline.evaluate(
    name='default_fs',
    model='lr', 
    pca=0
)

MRR: 0.81875
nDCG: 0.8823923863263119


##### Hyperparameter Optimization

In [8]:
logistic_regression_search_space: list = []
logistic_regression_search_space.append(Categorical(['l2', 'none'], name='penalty'))
logistic_regression_search_space.append(Real(0.1, 100.0, 'log-uniform', name='C'))
logistic_regression_search_space.append(Real(1e-6, 0.1, name='tol'))

In [9]:
pipeline.evaluate(
    name='hpo_fs',
    model='lr', 
    pca=0, 
    search_space=logistic_regression_search_space,
    trials=80
)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval

Best MRR: 0.7119841269841269
Best Hyperparameters: ['l2', 18.183153916405647, 0.06072346299946403]
MRR on test set: 0.81875
nDCG on test set: 0.8825771016778642


##### PCA

In [17]:
for i in range(2,27):
    pipeline.evaluate(
        name='pca_values_lr',
        model='lr', 
        pca=i
    )

MRR: 0.813421052631579
nDCG: 0.875380396548771
MRR: 0.816031746031746
nDCG: 0.8755223303437085
MRR: 0.816031746031746
nDCG: 0.8750046056822695
MRR: 0.816031746031746
nDCG: 0.8750884632570748
MRR: 0.815959595959596
nDCG: 0.8742662851688928
MRR: 0.8128129117259552
nDCG: 0.8739625009578208
MRR: 0.7477777777777779
nDCG: 0.8714715055789896
MRR: 0.7281196581196581
nDCG: 0.8695445794650032
MRR: 0.7440476190476191
nDCG: 0.8733379176318246
MRR: 0.7440476190476191
nDCG: 0.8733525898681468
MRR: 0.7440476190476191
nDCG: 0.8731938825606123
MRR: 0.7523809523809525
nDCG: 0.872082723509282
MRR: 0.7451587301587301
nDCG: 0.8711989041377475
MRR: 0.7451587301587301
nDCG: 0.8704416069939522
MRR: 0.7432539682539683
nDCG: 0.8707415848153173
MRR: 0.7452777777777777
nDCG: 0.8698594842774735
MRR: 0.7436111111111111
nDCG: 0.8700395120080298
MRR: 0.7781944444444444
nDCG: 0.8689749517819728
MRR: 0.7727777777777778
nDCG: 0.8670595433111249
MRR: 0.7727777777777778
nDCG: 0.8671854379256733
MRR: 0.7522222222222222
nDC

## Naive Bayes

In [10]:
pipeline.evaluate(
    name='default_fs',
    model='nbg', 
    pca=0
)

MRR: 0.7511111111111112
nDCG: 0.8686838174311546


##### PCA

In [18]:
for i in range(2,27):
    pipeline.evaluate(
        name='pca_values_nbg',
        model='nbg', 
        pca=i
    )

MRR: 0.8240123456790123
nDCG: 0.8719453663932762
MRR: 0.825952380952381
nDCG: 0.86914284033675
MRR: 0.8144971886351197
nDCG: 0.8366365854398193
MRR: 0.8144588744588745
nDCG: 0.8345095287486778
MRR: 0.8172749664128974
nDCG: 0.8345296498980742
MRR: 0.8172366522366522
nDCG: 0.8337468937626193
MRR: 0.7944832944832945
nDCG: 0.8493316687048718
MRR: 0.8047863247863247
nDCG: 0.8430748997242468
MRR: 0.7992063492063491
nDCG: 0.8357836673722847
MRR: 0.816056166056166
nDCG: 0.808070735018471
MRR: 0.8166666666666667
nDCG: 0.8056379404546669
MRR: 0.7806644880174292
nDCG: 0.804458273227931
MRR: 0.8037312360841773
nDCG: 0.8048786174465444
MRR: 0.8033496732026144
nDCG: 0.80289162422156
MRR: 0.7973972922502334
nDCG: 0.8013277995244635
MRR: 0.7814484126984127
nDCG: 0.8046023337560513
MRR: 0.7825641025641026
nDCG: 0.7993950328958814
MRR: 0.7993055555555556
nDCG: 0.8022305720578612
MRR: 0.7824252136752138
nDCG: 0.8009622236961355
MRR: 0.7588141025641025
nDCG: 0.8022485315039002
MRR: 0.7588141025641025
nDCG

## Multi-layer Perceptron

In [11]:
pipeline.evaluate(
    name='default_fs',
    model='mlp', 
    pca=0
)

MRR: 0.7409722222222223
nDCG: 0.8440657384344709


##### Hyperparameter Optimization

In [12]:
mlp_search_space: list = []
mlp_search_space.append(Categorical(['identity', 'logistic', 'tanh', 'relu'], name='activation'))
mlp_search_space.append(Real(1e-6, 0.1, name='alpha'))
mlp_search_space.append(Real(1e-6, 0.1, name='learning_rate_init'))

In [13]:
pipeline.evaluate(
    name='hpo_fs',
    model='mlp', 
    pca=0,
    search_space=mlp_search_space,
    trials=80
)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

Best MRR: 0.7562698412698411
Best Hyperparameters: ['tanh', 0.0064010012631932, 0.02845405860279831]
MRR on test set: 0.810103785103785
nDCG on test set: 0.8644829960193103


##### PCA

In [19]:
for i in range(2,27):
    pipeline.evaluate(
        name='pca_values_mlp',
        model='mlp', 
        pca=i
    )

MRR: 0.8019444444444445
nDCG: 0.8688720234348098
MRR: 0.812948717948718
nDCG: 0.8705786176321586
MRR: 0.8166666666666667
nDCG: 0.8684854136912821
MRR: 0.8190740740740741
nDCG: 0.8689959937205161
MRR: 0.8138888888888889
nDCG: 0.8513793614866301
MRR: 0.8186111111111111
nDCG: 0.8686583417791207
MRR: 0.7731196581196581
nDCG: 0.8601207149101547


/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MRR: 0.7233333333333333
nDCG: 0.8433781492010882


/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MRR: 0.704398148148148
nDCG: 0.8112909735335854
MRR: 0.7394444444444445
nDCG: 0.8497904675704727
MRR: 0.7265567765567765
nDCG: 0.8293091395014778
MRR: 0.7327777777777779
nDCG: 0.8343520763777875
MRR: 0.7219444444444444
nDCG: 0.8382131192246394


...

MRR: 0.6789285714285714
nDCG: 0.8024423242512796


/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MRR: 0.7061507936507937
nDCG: 0.7978066049033303


/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MRR: 0.5223450080515297
nDCG: 0.6571591256961091


## Support Vector Machine

In [18]:
pipeline.evaluate(
    name='default_fs',
    model='svm', 
    pca=0
)

MRR: 0.7314529914529916
nDCG: 0.8562510087514688


##### Hyperparameter Optimization

In [17]:
svm_search_space: list = []
svm_search_space.append(Categorical(['poly', 'rbf', 'sigmoid'], name='kernel'))
svm_search_space.append(Real(0.1, 100.0, name='C'))

In [18]:
pipeline.evaluate(
    name='hpo_fs',
    model='svm', 
    pca=0,
    search_space=svm_search_space,
    trials=80
)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Best MRR: 0.7601037851037851
Best Hyperparameters: ['rbf', 5.670025851035235]
MRR on test set: 0.6783730158730158
nDCG on test set: 0.8503540104234879


##### PCA

In [20]:
for i in range(2,27):
    pipeline.evaluate(
        name='pca_values_svm',
        model='svm', 
        pca=i
    )

MRR: 0.6433774250440917
nDCG: 0.7034037574857117
MRR: 0.6813139329805995
nDCG: 0.8089191590307245
MRR: 0.6634126984126983
nDCG: 0.7941898231002916
MRR: 0.6571075837742504
nDCG: 0.7972789412459317
MRR: 0.6614668856048167
nDCG: 0.8056809993783278
MRR: 0.6674641577060931
nDCG: 0.8080562059222445
MRR: 0.6056878306878307
nDCG: 0.8265873560062279
MRR: 0.6268518518518518
nDCG: 0.8255668285367399
MRR: 0.6293101343101344
nDCG: 0.831613397116967
MRR: 0.6556481481481481
nDCG: 0.8361573487842157
MRR: 0.6428703703703704
nDCG: 0.8349455867597315
MRR: 0.6178703703703704
nDCG: 0.8302699490871212
MRR: 0.6545370370370371
nDCG: 0.8330371510335718
MRR: 0.6457407407407406
nDCG: 0.8319489994037904
MRR: 0.6366137566137566
nDCG: 0.8306893379094372
MRR: 0.6268518518518519
nDCG: 0.829880292580937
MRR: 0.6223544973544972
nDCG: 0.8302003883607547
MRR: 0.6383974358974359
nDCG: 0.8297041028663922
MRR: 0.6383974358974359
nDCG: 0.8287793485206806
MRR: 0.6400641025641025
nDCG: 0.8286834844705233
MRR: 0.636730769230769

## Decision Tree

In [19]:
pipeline.evaluate(
    name='default_fs',
    model='dt', 
    pca=0
)

MRR: 0.5223240416386495
nDCG: 0.6197150517436344


##### Hyperparameter Optimization

In [19]:
decision_tree_search_space: list = []
decision_tree_search_space.append(Categorical(['gini', 'entropy'], name='criterion'))
decision_tree_search_space.append(Integer(2, 15, name='min_samples_split'))
decision_tree_search_space.append(Integer(1, 10, name='min_samples_leaf'))
decision_tree_search_space.append(Integer(5, 100, name='max_leaf_nodes'))
decision_tree_search_space.append(Integer(10, 50, name='max_depth'))
decision_tree_search_space.append(Real(0.0, 0.2, name='min_weight_fraction_leaf'))

In [20]:
pipeline.evaluate(
    name='hpo_fs',
    model='dt', 
    pca=0,
    search_space=decision_tree_search_space,
    trials=80
)

Best MRR: 0.7442201779701779
Best Hyperparameters: ['entropy', 15, 1, 93, 27, 0.0077977848774843485]
MRR on test set: 0.7425925925925925
nDCG on test set: 0.8160804156201119


##### PCA

In [21]:
for i in range(2,27):
    pipeline.evaluate(
        name='pca_values_dt',
        model='dt', 
        pca=i
    )

MRR: 0.5389500836694846
nDCG: 0.619691445293976
MRR: 0.6239763701925863
nDCG: 0.617882335316302
MRR: 0.2886995227292688
nDCG: 0.4368404167922325
MRR: 0.30206514089618486
nDCG: 0.45475554243758753
MRR: 0.2969875369431856
nDCG: 0.4486513690572141
MRR: 0.2101728069871374
nDCG: 0.42896945676825815
MRR: 0.10422464629855445
nDCG: 0.38229872044646634
MRR: 0.07047486706460022
nDCG: 0.3600218304063183
MRR: 0.08898674483080327
nDCG: 0.369263767097871
MRR: 0.13872500294806642
nDCG: 0.378788636055906
MRR: 0.07584884703057743
nDCG: 0.37606749868300476
MRR: 0.07073249770597348
nDCG: 0.37760938834292546
MRR: 0.06445534234454815
nDCG: 0.3929215469079622
MRR: 0.06050407746975946
nDCG: 0.36588702313757737
MRR: 0.10552820415358097
nDCG: 0.3780082854106514
MRR: 0.05635410937777335
nDCG: 0.3690665969034469
MRR: 0.049967708765519495
nDCG: 0.3645722961134934
MRR: 0.08639846622609643
nDCG: 0.3746560916119273
MRR: 0.07021487839793869
nDCG: 0.372036138201669
MRR: 0.18823376641655784
nDCG: 0.40372931649978355
MR

## Random Forest

In [20]:
pipeline.evaluate(
    name='default_fs',
    model='rf', 
    pca=0
)

MRR: 0.6753174603174603
nDCG: 0.8062385343678714


##### Hyperparameter Optimization

In [22]:
random_forest_search_space: list = []
random_forest_search_space.append(Integer(20, 200, name='n_estimators'))
random_forest_search_space.append(Categorical(['gini', 'entropy'], name='criterion'))
random_forest_search_space.append(Integer(2, 15, name='min_samples_split'))
random_forest_search_space.append(Integer(1, 10, name='min_samples_leaf'))
random_forest_search_space.append(Integer(5, 100, name='max_leaf_nodes'))
random_forest_search_space.append(Integer(10, 50, name='max_depth'))
random_forest_search_space.append(Real(0.0, 0.2, name='min_weight_fraction_leaf'))

In [23]:
pipeline.evaluate(
    name='hpo_fs',
    model='rf', 
    pca=0,
    search_space=random_forest_search_space,
    trials=80
)

Best MRR: 0.80140522875817
Best Hyperparameters: [94, 'entropy', 15, 2, 40, 33, 0.19949999886994965]
MRR on test set: 0.6867307692307693
nDCG on test set: 0.8373044508822045


##### PCA

In [22]:
for i in range(2,27):
    pipeline.evaluate(
        name='pca_values_rf',
        model='rf', 
        pca=i
    )

MRR: 0.7096590708355415
nDCG: 0.7385102104159841
MRR: 0.7190291620524178
nDCG: 0.7494594608606863
MRR: 0.6788168724279835
nDCG: 0.7378862037038021
MRR: 0.6653968253968255
nDCG: 0.7401299725431201
MRR: 0.6718949771689499
nDCG: 0.7428420876063935
MRR: 0.7859977324263039
nDCG: 0.750654937893532
MRR: 0.6972222222222223
nDCG: 0.7928258993314393
MRR: 0.7430555555555557
nDCG: 0.7869403209831035
MRR: 0.6749206349206349
nDCG: 0.780668165338073
MRR: 0.7248821548821548
nDCG: 0.7805781514834106
MRR: 0.7102777777777778
nDCG: 0.7883097225981518
MRR: 0.6905303030303032
nDCG: 0.7745890690562053
MRR: 0.7251089324618737
nDCG: 0.7785301054021366
MRR: 0.6331878306878307
nDCG: 0.7682678501747221
MRR: 0.6073015873015872
nDCG: 0.7612619966886485
MRR: 0.642952847805789
nDCG: 0.7701403776985004
MRR: 0.7260042735042734
nDCG: 0.78042275673895
MRR: 0.6364285714285713
nDCG: 0.7713464231245808
MRR: 0.7022895622895623
nDCG: 0.7770028115071369
MRR: 0.7073648573648573
nDCG: 0.7787323352992446
MRR: 0.6128338001867414
n

## Adaptive Boosting

In [21]:
pipeline.evaluate(
    name='default_fs',
    model='ada', 
    pca=0
)

MRR: 0.8146031746031746
nDCG: 0.8598276886566136


##### Hyperparameter Optimization

In [9]:
ada_boost_search_space: list = []
ada_boost_search_space.append(Integer(20, 200, name='n_estimators'))
ada_boost_search_space.append(Real(1e-2, 2.0, name='learning_rate'))

In [11]:
pipeline.evaluate(
    name='hpo_fs',
    model='ada', 
    pca=0,
    search_space=ada_boost_search_space,
    trials=80
)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/skopt/optimizer/optimize

Best MRR: 0.7508080808080808
Best Hyperparameters: [167, 0.01]
MRR on test set: 0.6775396825396826
nDCG on test set: 0.8332631471987071


##### PCA

In [ ]:
for i in range(2,27):
    pipeline.evaluate(
        name='pca_values_ada',
        model='ada', 
        pca=i
    )

MRR: 0.767070707070707
nDCG: 0.8505790620936556
MRR: 0.7209595959595961
nDCG: 0.8465075653552494
MRR: 0.7870370370370371
nDCG: 0.8533416829303844
MRR: 0.7623456790123456
nDCG: 0.8525416614984044
MRR: 0.7367543859649123
nDCG: 0.8471600834550975
MRR: 0.7947846889952153
nDCG: 0.8591390617858541
MRR: 0.6580952380952382
nDCG: 0.8465280527965331
MRR: 0.7604166666666665
nDCG: 0.8602537461789603
MRR: 0.7464052287581701
nDCG: 0.8579250527493678
MRR: 0.65515522875817
nDCG: 0.8513746659725856
MRR: 0.7991830065359476
nDCG: 0.8626374942665815
MRR: 0.729861111111111
nDCG: 0.8541247827167482
MRR: 0.7433333333333331
nDCG: 0.8530896976079126
MRR: 0.7283333333333332
nDCG: 0.8518228766280074
MRR: 0.7283333333333332
nDCG: 0.8518621693033482
MRR: 0.6588888888888889
nDCG: 0.8010308778709107
MRR: 0.6588888888888889
nDCG: 0.8010308778709107
MRR: 0.7053174603174602
nDCG: 0.8447397820293783
MRR: 0.7053174603174602
nDCG: 0.8447397820293783
MRR: 0.72375
nDCG: 0.8490602069350033


## Gradient Boosting

In [22]:
pipeline.evaluate(
    name='default_fs',
    model='gb', 
    pca=0
)

MRR: 0.7280440685045948
nDCG: 0.8254641892372658


##### Hyperparameter Optimization

In [15]:
gradient_boosting_search_space: list = []
gradient_boosting_search_space.append(Integer(20, 200, name='n_estimators'))
gradient_boosting_search_space.append(Categorical(['friedman_mse', 'squared_error'], name='criterion'))
gradient_boosting_search_space.append(Integer(2, 15, name='min_samples_split'))
gradient_boosting_search_space.append(Integer(1, 10, name='min_samples_leaf'))
gradient_boosting_search_space.append(Integer(5, 100, name='max_leaf_nodes'))
gradient_boosting_search_space.append(Integer(10, 50, name='max_depth'))
gradient_boosting_search_space.append(Real(0.0, 0.2, name='min_weight_fraction_leaf'))

In [16]:
pipeline.evaluate(
    name='hpo_fs',
    model='gb', 
    pca=0,
    search_space=gradient_boosting_search_space,
    trials=80
)

Best MRR: 0.7750348092453355
Best Hyperparameters: [143, 'friedman_mse', 2, 7, 90, 29, 0.029847021027296184]
MRR on test set: 0.6677777777777777
nDCG on test set: 0.8274833154070339


##### PCA

In [5]:
for i in range(2,27):
    pipeline.evaluate(
        name='pca_values_gb',
        model='gb', 
        pca=i
    )

MRR: 0.5547981779217477
nDCG: 0.6998194679721532
MRR: 0.8112820512820512
nDCG: 0.816040309597916
MRR: 0.742008547008547
nDCG: 0.7697241792167222
MRR: 0.7588888888888888
nDCG: 0.790320690472736
MRR: 0.7758080808080808
nDCG: 0.7895205117698157
MRR: 0.7198148148148148
nDCG: 0.7638664399158597
MRR: 0.6901785714285714
nDCG: 0.7895532277520101
MRR: 0.7647619047619048
nDCG: 0.7969533530775652
MRR: 0.7196230158730159
nDCG: 0.7786377256900336
MRR: 0.7252645502645502
nDCG: 0.7755667753017924
MRR: 0.7627492877492876
nDCG: 0.7680283861601982
MRR: 0.7342261904761905
nDCG: 0.7780718301887978
MRR: 0.7276750700280111
nDCG: 0.7808640260868638
MRR: 0.7134920634920635
nDCG: 0.7739940260306952
MRR: 0.693949275362319
nDCG: 0.769446357682961
MRR: 0.6918452380952381
nDCG: 0.7778657054503009
MRR: 0.7047817460317459
nDCG: 0.7697177293293085
MRR: 0.6676851851851853
nDCG: 0.7482967623233314
MRR: 0.6951851851851851
nDCG: 0.7531432839604523
MRR: 0.6697121070650482
nDCG: 0.7498673056395885
MRR: 0.6943184885290149
n